In [1]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import skimage.transform

In [2]:
def get_filelist(img_loc):
    filelist = glob.glob(img_loc+'*.jpg')
    filelist.sort()
    return filelist

In [3]:
img_loc = '../ISIC2018_Task3_Training_Input/'

filelist = get_filelist(img_loc)

In [24]:
# Load images as arrays

bs = len(filelist)
input_shape = [bs, 224, 224, 3]
input_shape_inception = [bs, 299, 299, 3]
num_crops = 4

def get_final_img_arrs(filelist, input_shape, Y_df, num_crops):
    
    bs, n_H, n_W, n_C = input_shape
    bs_rc = (np.sum((Y_df['DF']==1).astype(int)) + np.sum((Y_df['VASC']==1).astype(int)))*num_crops
    X_rc = np.empty([bs_rc, n_H, n_W, n_C])
    X_orig = np.empty([bs, n_H, n_W, n_C])
    bs, n_H, n_W, n_C = input_shape
    Xrcloc = 0
    for i, fname in enumerate(filelist[:bs]):
        img = plt.imread(fname)
        if int(Y_df.iloc[i][6]) == 1 or int(Y_df.iloc[i][7]) == 1:
            for j in range(num_crops):
                c = np.random.choice(range(-20,20),4)
                img_cropped = img[50+c[0]:400+c[1],50+c[2]:550+c[3],:]
                img_cropped = skimage.transform.resize(img, (n_H,n_W), mode='constant')
                X_rc[Xrcloc] = img_cropped
                Xrcloc +=1
        img = skimage.transform.resize(img, (n_H,n_W), mode='constant')
        X_orig[i] = img
        if i%100 == 99 or i == bs-1:
            print('{} files loaded'.format(i+1))
    X_final = np.vstack([X_orig, X_rc])
    
    return X_final

In [5]:
labels_loc = '../ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth.csv'
Y_df = pd.read_csv(labels_loc)

In [25]:
X_final = get_final_img_arrs(filelist, input_shape, Y_df, num_crops)

/scratch/user/pdwarkanath/.conda/envs/myenv3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


100 files loaded
200 files loaded
300 files loaded
400 files loaded
500 files loaded
600 files loaded
700 files loaded
800 files loaded
900 files loaded
1000 files loaded
1100 files loaded
1200 files loaded
1300 files loaded
1400 files loaded
1500 files loaded
1600 files loaded
1700 files loaded
1800 files loaded
1900 files loaded
2000 files loaded
2100 files loaded
2200 files loaded
2300 files loaded
2400 files loaded
2500 files loaded
2600 files loaded
2700 files loaded
2800 files loaded
2900 files loaded
3000 files loaded
3100 files loaded
3200 files loaded
3300 files loaded
3400 files loaded
3500 files loaded
3600 files loaded
3700 files loaded
3800 files loaded
3900 files loaded
4000 files loaded
4100 files loaded
4200 files loaded
4300 files loaded
4400 files loaded
4500 files loaded
4600 files loaded
4700 files loaded
4800 files loaded
4900 files loaded
5000 files loaded
5100 files loaded
5200 files loaded
5300 files loaded
5400 files loaded
5500 files loaded
5600 files loaded
5

In [26]:
X_final.shape

(11043, 224, 224, 3)

In [27]:
def get_final_img_labels(Y_df, input_shape, num_crops):
    Y_df_rc = pd.DataFrame(columns = Y_df.columns)
    bs, n_H, n_W, n_C = input_shape
    Ydfrcloc = 0
    for i, fname in enumerate(filelist[:bs]):
        if int(Y_df.iloc[i][6]) == 1 or int(Y_df.iloc[i][7]) == 1:
            for j in range(num_crops):
                Y_df_rc.loc[Ydfrcloc] = Y_df.iloc[i]
                Ydfrcloc +=1
    Y_final_df = pd.concat([Y_df, Y_df_rc], ignore_index=True)
    return Y_final_df

In [29]:
Y_final_df = get_final_img_labels(Y_df, input_shape, num_crops)
Y_final_df.shape

(11043, 8)

In [30]:
X_final_file = 'X_final.npy'
np.save(X_final_file, X_final)

#X_orig_inception_file = 'X_orig_inception.npy'
#np.save(X_orig_inception_file, X_orig_inception)

In [31]:
Y_final_df.to_csv('Y_final_df.csv', index = False)

In [34]:
class_weights = np.array(np.mean(Y_final_df.iloc[:, 1:], axis = 0)).astype('float32')

In [35]:
class_weights

array([0.10078783, 0.60717195, 0.04654532, 0.02961152, 0.09952006,
       0.05206918, 0.06429412], dtype=float32)